# Install and import libraries

In [1]:
#!pip install yfinance

In [2]:
import pandas as pd
import numpy as np

import yfinance as yf

# Load, exlore and clean data

In [3]:
tickers = ['AAPL', 'MSFT', 'AMZN', 'GOOG', 'META', 'TSLA', 'NVDA', 'PYPL', 'ADBE', 'NFLX']

In [4]:
stock_price_history = yf.download(tickers, start="2010-01-01", end="2020-12-31")

[*********************100%%**********************]  10 of 10 completed


In [5]:
stock_price_history.head()

Adj Close                                                          \
                AAPL       ADBE    AMZN       GOOG META       MSFT      NFLX   
Date                                                                           
2010-01-04  6.478997  37.090000  6.6950  15.610239  NaN  23.522564  7.640000   
2010-01-05  6.490200  37.700001  6.7345  15.541497  NaN  23.530169  7.358571   
2010-01-06  6.386965  37.619999  6.6125  15.149715  NaN  23.385761  7.617143   
2010-01-07  6.375158  36.889999  6.5000  14.797037  NaN  23.142563  7.485714   
2010-01-08  6.417542  36.689999  6.6760  14.994298  NaN  23.302166  7.614286   

                                ...     Volume                                 \
                NVDA PYPL TSLA  ...       AAPL     ADBE       AMZN       GOOG   
Date                            ...                                             
2010-01-04  4.240801  NaN  NaN  ...  493729600  4710200  151998000   78541293   
2010-01-05  4.302728  NaN  NaN  ...  601904800  7108800  177038000  120638494   
2010-01-06  4.330253  NaN  NaN  ...  552160000  5336400  143576000  159744526   
2010-01-07  4.245390  NaN  NaN  ...  477131200  5576700  220604000  257533695   
2010-01-08  4.254563  NaN  NaN  ...  447610800  5429200  196610000  189680313   

                                                         
           META      MSFT      NFLX      NVDA PYPL TSLA  
Date                                                     
2010-01-04  NaN  38409100  17239600  80020400  NaN  NaN  
2010-01-05  NaN  49749600  23753100  72864800  NaN  NaN  
2010-01-06  NaN  58182400  23290400  64916800  NaN  NaN  
2010-01-07  NaN  50559700   9955400  54779200  NaN  NaN  
2010-01-08  NaN  51197400   8180900  47816800  NaN  NaN  

[5 rows x 60 columns]

In [6]:
df = stock_price_history["Adj Close"].reset_index()

In [7]:
df.fillna(method='ffill', inplace=True) # forward fill
df.fillna(0, inplace=True) # fill the rest with 0 (beginning of the time series)

# Prepare train and test dataset

In [8]:
test_size = int(0.2*len(df)) # 20% test size
train_df = df[:-test_size]
test_df = df[-test_size:]

train_df.shape, test_df.shape

((2215, 11), (553, 11))

# Train models

In [9]:
from prophet import Prophet

In [10]:
# create for loop to train prophet model for each ticker
models = {}

for ticker in tickers:
    model = Prophet()
    model.fit(train_df[["Date", ticker]].rename(columns={"Date": "ds", ticker: "y"}))
    models[ticker] = model

c:\Users\marek.stepan\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
22:17:17 - cmdstanpy - INFO - Chain [1] start processing
22:17:17 - cmdstanpy - INFO - Chain [1] done processing
22:17:17 - cmdstanpy - INFO - Chain [1] start processing
22:17:19 - cmdstanpy - INFO - Chain [1] done processing
22:17:19 - cmdstanpy - INFO - Chain [1] start processing
22:17:20 - cmdstanpy - INFO - Chain [1] done processing
22:17:20 - cmdstanpy - INFO - Chain [1] start processing
22:17:20 - cmdstanpy - INFO - Chain [1] done processing
22:17:20 - cmdstanpy - INFO - Chain [1] start processing
22:17:21 - cmdstanpy - INFO - Chain [1] done processing
22:17:21 - cmdstanpy - INFO - Chain [1] start processing
22:17:22 - cmdstanpy - INFO - Chain [1] done processing
22:17:22 - cmdstanpy - INFO - Chain [1] start processing
22:17

# Explore predictions

In [11]:
from prophet.plot import plot_plotly, plot_components_plotly

In [12]:
model = models[tickers[0]]

future = model.make_future_dataframe(periods=test_size) # create ds for future dates
forecast = model.predict(future) # predict y for future dates

In [13]:
plot_plotly(model, forecast)

In [14]:
plot_components_plotly(model, forecast)

# Evaluate predictions on test set

In [15]:
# import mse
from sklearn.metrics import mean_squared_error
import numpy as np

In [16]:
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

In [17]:
armse_scores = {}
rrmse_scores = {}

for ticker in tickers:
    y_true = test_df[ticker]

    model = models[ticker]
    future = model.make_future_dataframe(periods=test_size)
    forecast = model.predict(future)
    y_pred = forecast.iloc[-test_size:]["yhat"]

    armse_scores[ticker] = np.round(rmse(y_true, y_pred),3)
    rrmse_scores[ticker] = np.round(100*armse_scores[ticker]/y_true.mean(), 2)

In [18]:
results = pd.DataFrame([armse_scores, rrmse_scores], index=["Absolute RMSE", "Relative RMSE"]).T.sort_values("Relative RMSE")
results

,Absolute RMSE,Relative RMSE
GOOG,8.090,12.36
ADBE,49.689,14.71
META,34.838,17.24
AMZN,21.352,19.60
MSFT,35.180,23.25
PYPL,33.078,25.68
NFLX,126.941,33.54
AAPL,24.305,35.02
NVDA,33.869,49.49
TSLA,61.499,113.67
